In [1]:
# Import packages
from collections import Counter
import csv
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacytextblob.spacytextblob import SpacyTextBlob
import time
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams.update({"font.size": 12})

# 1. Load Dataset

In [40]:
submissions_csv = "data/submissions.csv"
comments_csv = "data/comments_pratik.csv"
feature_comments_csv = "data/feature_comments_rakib_new.csv"

date_fmt = lambda x: datetime.utcfromtimestamp(int(x)).strftime("%Y-%m-%d")

## 1.1. Load submissions data

In [38]:
# Load submissions data
df_submissions = pd.read_csv(submissions_csv, delimiter=";", parse_dates=["created_utc"],
                 date_parser=date_fmt, encoding="utf-8")
df_submissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197819 entries, 0 to 1197818
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   author                 1197819 non-null  object        
 1   created_utc            1197819 non-null  datetime64[ns]
 2   id                     1197819 non-null  object        
 3   link_flair_text        1145788 non-null  object        
 4   num_comments           1197819 non-null  int64         
 5   removed_by_category    897146 non-null   object        
 6   score                  1197819 non-null  int64         
 7   selftext               693501 non-null   object        
 8   subreddit_subscribers  1197819 non-null  int64         
 9   title                  1197818 non-null  object        
 10  upvote_ratio           1084007 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 100.5+ MB


In [29]:
df_submissions[["id","created_utc", "author","link_flair_text","title","selftext","num_comments","score",
                "upvote_ratio","removed_by_category"]].tail(1000)[40:45]

,id,created_utc,author,link_flair_text,title,selftext,num_comments,score,upvote_ratio,removed_by_category
1196859,mh96r3,2021-03-31,crockett5,Discussion,"A poor Ape doesn't have to always be retarded,...",I'm not trying to give financial advice for du...,13,1,1.0,NaN
1196860,mh96t7,2021-03-31,Madmonkey45,News,Could we see another surge in pot stocks as NY...,NaN,2,1,1.0,moderator
1196861,mh96wm,2021-03-31,BRENBRENBRENBREN1,News,Genius brands,Is genius brands a good buy?,3,1,1.0,NaN
1196862,mh970t,2021-03-31,Leetomnsx,Discussion,🦍 is getting ready to go buy a bank to deposit...,NaN,0,1,1.0,NaN
1196863,mh97ec,2021-03-31,allintraders,YOLO,Letssss Gooooooooo!!!!!! Hertz!,NaN,1,1,1.0,NaN


## 1.2. Load comments data

In [41]:
df_comments = pd.read_csv(comments_csv, delimiter=";", parse_dates=["created_utc"],
                date_parser=date_fmt, encoding="utf-8")
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6501377 entries, 0 to 6501376
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   author       object        
 1   body         object        
 2   created_utc  datetime64[ns]
 3   id           object        
 4   link_id      object        
 5   score        int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 297.6+ MB


In [35]:
df_comments[["id","created_utc","author","body","score","link_id"]].head(20)[8:13]

,id,created_utc,author,body,score,link_id
8,g90aae4,2020-10-16,0laugh,Shits gonna drop to like $78. Then load up on ...,3,t3_jcbasm
9,g90af9u,2020-10-16,DiepioHybrid,I would if I hadn't dropped my last $50 on thi...,1,t3_jcbasm
10,g90akra,2020-10-16,mechanic-panic,"I bought in at $95 after it crashed, I thought...",2,t3_jcbasm
11,g90amr5,2020-10-16,askingforafavor12345,Gonna drop to mid $70s and slowly recover over...,2,t3_jcbasm
12,g90b3j0,2020-10-16,0laugh,75-79 will be it's support then sky rocket again.,2,t3_jcbasm


## 1.3. Load S&P 500 tickers

In [5]:
# Load S&P 500 tickers
with open("data/sp500-symbol-list.txt") as tickers:
    sp500 = [ticker.rstrip() for ticker in tickers]

## 1.4. Load non S&P 500 tickers

In [6]:
# Load non S&P 500 tickers
nasdaq_stocks = pd.read_csv("data/nasdaq_stocks.csv", sep=",")
nasdaq_stocks_symbols = nasdaq_stocks["Symbol"].tolist()
non_sp500 = list(set(nasdaq_stocks_symbols) - set(sp500))

## 1.5. Load lingos

In [7]:
# Walstreetbets lingo ref: https://economictimes.indiatimes.com/markets/stocks/news/from-stonks-to-bagholder-deciphering-reddits-trading-lingo
lingo = ["YOLO", "BAGHOLDER", "TENDIES", "DD", "GO", "BUY", "BUYING", "buying", "go", "holding", "high", 
         "moon", "squeeze"]

# 2. Preprocess Data

## 2.1. Filter removed submissions

Remove submissions that are deleted by author or moderator or Reddit filter.
- **anti_evil_ops, author, deleted**: Post was removed by author.
- **automod_filtered**: Post is awaiting moderator approval.
- **moderator**: Post was removed by moderator.
- **reddit**: Post was removed by Reddit's spam filters.

In [8]:
df_submissions = df_submissions[df_submissions["removed_by_category"].isnull()]
df_submissions.drop("removed_by_category", axis=1, inplace=True)
df_submissions.head()

,author,created_utc,id,link_flair_text,num_comments,score,selftext,subreddit_subscribers,title,upvote_ratio
3,ILL3NITVM,2020-01-01,eib3bh,NaN,0,1,"Happy new year to everyone, stay hydrated &amp...",772368,HAPPY NEW YEAR!,NaN
5,theOriginalTurd,2020-01-01,eib3w8,Discussion,18,1,"Yesterday I bought 130 shares of ACB, and toda...",772368,Cannabis Industry Bull Run,NaN
8,winkerpack,2020-01-01,eibdob,NaN,28,1,What better way to bring in 2020 than with a G...,772380,GUH! Your way into the New Year! Time your 202...,NaN
9,EagleInvestors,2020-01-01,eibfcr,YOLO,17,1,Just as the title implies.. What would you cuc...,772386,Which Stonk is primed for a YOLO?,NaN
10,Swipe4Swipes,2020-01-01,eibn2v,NaN,0,1,NaN,772399,My fellow stock traders ! Do you guys prefer t...,NaN


## 2.2. Filter removed comments

Remove comments associated with removed submissions. Only keep comments from non-removed submissions.

In [51]:
df_comments = df_comments[df_comments["author"] != "[deleted]"]
df_comments = df_comments[~df_comments["created_utc"].isna()]

In [10]:
df_comments["parent_id"] = df_comments["link_id"].apply(lambda x: x.split("_")[1])
df_comments.head()

,author,body,created_utc,id,link_id,score,parent_id
1,ADONIS_VON_MEGADONG,I already got in but decided to say YOLO and s...,2021-01-28,gl35t1p,t3_l6x2gv,3,l6x2gv
3,ADONIS_VON_MEGADONG,Yeah the thing is I actually have some faith i...,2021-01-28,gl377lu,t3_l6x2gv,2,l6x2gv
7,sf9191,I’ve been with them for two years and have hat...,2021-01-28,gl389xu,t3_l6x2hd,1,l6x2hd
9,amcneel,BB,2021-01-28,gl35j7i,t3_l6x2hj,6,l6x2hj
15,MineIsLongerThanYour,Not now man. Focus on the fucking gme,2021-01-28,gl35kwp,t3_l6x2ho,3,l6x2ho


In [11]:
# Do inner join which will remove the comments whose id is not in submissions dataframe
df_comments = pd.merge(df_comments, df_submissions, left_on="parent_id", right_on="id", how="inner")
df_comments.head()

,author_x,body,created_utc_x,id_x,link_id,score_x,parent_id,author_y,created_utc_y,id_y,link_flair_text,num_comments,score_y,selftext,subreddit_subscribers,title,upvote_ratio
0,ADONIS_VON_MEGADONG,I already got in but decided to say YOLO and s...,2021-01-28,gl35t1p,t3_l6x2gv,3,l6x2gv,ADONIS_VON_MEGADONG,2021-01-28,l6x2gv,Discussion,4,1,I set an order for some additional shares of B...,4517630,Just FYI Robinhood is restricting the sale of ...,1.0
1,ADONIS_VON_MEGADONG,Yeah the thing is I actually have some faith i...,2021-01-28,gl377lu,t3_l6x2gv,2,l6x2gv,ADONIS_VON_MEGADONG,2021-01-28,l6x2gv,Discussion,4,1,I set an order for some additional shares of B...,4517630,Just FYI Robinhood is restricting the sale of ...,1.0
2,sf9191,I’ve been with them for two years and have hat...,2021-01-28,gl389xu,t3_l6x2hd,1,l6x2hd,sf9191,2021-01-28,l6x2hd,Discussion,2,1,NaN,4517657,Ally trading platform appears to allow GME tra...,1.0
3,amcneel,BB,2021-01-28,gl35j7i,t3_l6x2hj,6,l6x2hj,auldwardog,2021-01-28,l6x2hj,NaN,7,1,Can we as a group replicate what happened with...,4517657,"AMC, NOK, BB - which is next??",1.0
4,MineIsLongerThanYour,Not now man. Focus on the fucking gme,2021-01-28,gl35kwp,t3_l6x2ho,3,l6x2ho,Abject_Opportunity35,2021-01-28,l6x2ho,Discussion,1,1,NaN,4517657,BUY AND HOLD $NOK WE HAVE TO ALL COME TOGETHER🚀,1.0


In [12]:
# Create new comments dataframe
df_comments = df_comments[["author_x", "body", "created_utc_x", "id_x", "score_x", "parent_id"]]
df_comments.head()

,author_x,body,created_utc_x,id_x,score_x,parent_id
0,ADONIS_VON_MEGADONG,I already got in but decided to say YOLO and s...,2021-01-28,gl35t1p,3,l6x2gv
1,ADONIS_VON_MEGADONG,Yeah the thing is I actually have some faith i...,2021-01-28,gl377lu,2,l6x2gv
2,sf9191,I’ve been with them for two years and have hat...,2021-01-28,gl389xu,1,l6x2hd
3,amcneel,BB,2021-01-28,gl35j7i,6,l6x2hj
4,MineIsLongerThanYour,Not now man. Focus on the fucking gme,2021-01-28,gl35kwp,3,l6x2ho


# 3. Feature Engineering

## 3.1 Submissions

### 3.1.1. Calculate sentiment per day

In [13]:
nlp = spacy.blank("en")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")
    nlp.rename_pipe("spacytextblob", "sentiment")

if "emoji" not in nlp.pipe_names:
    nlp.add_pipe("emoji")
    
print(nlp.pipe_names)

['sentiment', 'emoji']


In [14]:
df_submissions["title_polarity"] = 0.0
df_submissions["title_subjectivity"] = 0.0
df_submissions["body_polarity"] = 0.0
df_submissions["body_subjectivity"] = 0.0
df_submissions["emojis"] = ""

def get_submission_sentiment(title, body):
    nlp_title = nlp(str(title))
    nlp_body = nlp(str(body))
    emojis = nlp_title._.emoji + nlp_body._.emoji
    emojis = [emoji[0] for emoji in emojis]
    return pd.Series([nlp_title._.polarity, nlp_title._.subjectivity,
                      nlp_body._.polarity, nlp_body._.subjectivity, emojis])

df_submissions[["title_polarity", "title_subjectivity", "body_polarity", "body_subjectivity", "emojis"]] = df_submissions.apply(lambda row: get_submission_sentiment(row["title"], row["selftext"]), axis=1)
df_submissions.head()

In [15]:
# # Group by date
df_submissions_title_subjectivity = df_submissions.groupby(["created_utc"])["title_subjectivity"].sum().reset_index()

df_submissions_body_polarity = df_submissions.groupby(["created_utc"])["body_polarity"].sum().reset_index()
df_submissions_body_subs_title_polarity = df_submissions.groupby(["created_utc"])["title_polarity"].sum().reset_index()
df_submissionjectivity = df_submissions.groupby(["created_utc"])["body_subjectivity"].sum().reset_index()

### 3.1.2. Calculate emojis per day

In [16]:
def sanitize_emojis(data):
    return data.replace("[", "").replace("]", "").replace(",", "").replace("'", "").replace(" ", "")

In [17]:
df_submissions_emojis = df_submissions.groupby(["created_utc"]).apply(lambda x: str(list(x["emojis"]))).to_frame("emojis").reset_index()
df_submissions_emojis["emojis"] = df_submissions_emojis["emojis"].apply(sanitize_emojis)
df_submissions_emojis["emojis"] = df_submissions_emojis.apply(lambda x: dict(Counter(x["emojis"])), axis=1)

### 3.1.3. Calculate mentioned stocks and lingos count per day

In [18]:
# Function to find stocks in a text given stock list
regex = re.compile("[^a-zA-Z ]")
sp500_set = set(sp500)
non_sp500_set = set(non_sp500)
lingo_set = set(lingo)

def calculate_mentioned_stocks(title, body=""):
    content = regex.sub("", str(title) + " " + str(body)).split(" ")
    content = set(content)
    words_sp500 = str(list(sp500_set & content))
    words_non_sp500 = str(list(non_sp500_set & content))
    
    if len(words_sp500) > 2:
        lingos_sp500 = str(list(lingo_set & content))
    else:
        lingos_sp500 = str([])
    
    if len(words_non_sp500) > 2:
        lingos_non_sp500 = str(list(lingo_set & content))
    else:
        lingos_non_sp500 = str([])
    
    return pd.Series([words_sp500, words_non_sp500, lingos_sp500, lingos_non_sp500])

In [19]:
df_submissions[["sp500_stocks", "non_sp500_stocks", "sp500_lingos", "non_sp500_lingos"]] = df_submissions.apply(lambda x: calculate_mentioned_stocks(x["title"], x["selftext"]), axis=1)
df_submissions.head()

In [20]:
# # Group by date
df_submissions_sp500_stocks = df_submissions.groupby(["created_utc"])["sp500_stocks"].apply(",".join).reset_index()
df_submissions_non_sp500_stocks = df_submissions.groupby(["created_utc"])["non_sp500_stocks"].apply(",".join).reset_index()

df_submissions_sp500_lingos = df_submissions.groupby(["created_utc"])["sp500_lingos"].apply(",".join).reset_index()
df_submissions_non_sp500_lingos = df_submissions.groupby(["created_utc"])["non_sp500_lingos"].apply(",".join).reset_index()

In [21]:
def sanitize_column(column):
    column = column.replace("[],", "").replace("[]", "").replace("],[", ",")
    return column

In [22]:
# # Sanitize data
df_submissions_sp500_stocks["sp500_stocks"] = df_submissions_sp500_stocks["sp500_stocks"].apply(sanitize_column)
df_submissions_non_sp500_stocks["non_sp500_stocks"] = df_submissions_non_sp500_stocks["non_sp500_stocks"].apply(sanitize_column)

df_submissions_sp500_lingos["sp500_lingos"] = df_submissions_sp500_lingos["sp500_lingos"].apply(sanitize_column)
df_submissions_non_sp500_lingos["non_sp500_lingos"] = df_submissions_non_sp500_lingos["non_sp500_lingos"].apply(sanitize_column)

In [23]:
# # Count stocks and lingos
df_submissions_sp500_stocks["sp500_stocks"] = df_submissions_sp500_stocks["sp500_stocks"].str.strip("[]").str.split(",")
df_submissions_sp500_stocks["sp500_stocks"] = df_submissions_sp500_stocks.apply(lambda x: dict(Counter(x['sp500_stocks'])), axis=1)

df_submissions_non_sp500_stocks["non_sp500_stocks"] = df_submissions_non_sp500_stocks["non_sp500_stocks"].str.strip("[]").str.split(",")
df_submissions_non_sp500_stocks["non_sp500_stocks"] = df_submissions_non_sp500_stocks.apply(lambda x: dict(Counter(x["non_sp500_stocks"])), axis=1)

df_submissions_sp500_lingos["sp500_lingos"] = df_submissions_sp500_lingos["sp500_lingos"].str.strip("[]").str.split(",")
df_submissions_sp500_lingos["sp500_lingos"] = df_submissions_sp500_lingos.apply(lambda x: dict(Counter(x["sp500_lingos"])), axis=1)

df_submissions_non_sp500_lingos["non_sp500_lingos"] = df_submissions_non_sp500_lingos["non_sp500_lingos"].str.strip("[]").str.split(",")
df_submissions_non_sp500_lingos["non_sp500_lingos"] = df_submissions_non_sp500_lingos.apply(lambda x: dict(Counter(x["non_sp500_lingos"])), axis=1)

### 3.1.4. Count per day

In [24]:
df_submissions_count = df_submissions.groupby(["created_utc"])["title"].count().reset_index()
df_submissions_count.rename(columns={"title": "count"}, inplace=True)

### 3.1.5. Calculate total scores per day

In [25]:
df_submissions_score = df_submissions.groupby(["created_utc"])["score"].sum().reset_index()

### 3.1.6. Calculate total upvote ratio per day

In [26]:
df_submissions_upvote_ratio = df_submissions.groupby(["created_utc"])["upvote_ratio"].sum().reset_index()

### 3.1.7. Calculate flair category counts per day

In [27]:
def sanitize_category(df):
    category_dict = dict(Counter(df["category"]))
    if np.nan in category_dict.keys():
        category_dict["others"] = category_dict.pop(np.nan)
    return category_dict

df_submissions_category = df_submissions.groupby(["created_utc"]).apply(lambda x: list(x["link_flair_text"])).to_frame("category").reset_index()
df_submissions_category["category"] = df_submissions_category.apply(lambda x: sanitize_category(x), axis=1)

### 3.1.8. Create the final submissions dataframe

In [28]:
df_submissions_daily = pd.concat([df_submissions_sp500_stocks["created_utc"], 
                                  df_submissions_title_polarity["title_polarity"], 
                                  df_submissions_title_subjectivity["title_subjectivity"], 
                                  df_submissions_body_polarity["body_polarity"], 
                                  df_submissions_body_subjectivity["body_subjectivity"], 
                                  df_submissions_count["count"],
                                  df_submissions_score["score"],
                                  df_submissions_upvote_ratio["upvote_ratio"],
                                  df_submissions_emojis["emojis"],
                                  df_submissions_category["category"],
                                  df_submissions_sp500_stocks["sp500_stocks"], 
                                  df_submissions_non_sp500_stocks["non_sp500_stocks"],
                                  df_submissions_sp500_lingos["sp500_lingos"], 
                                  df_submissions_non_sp500_lingos["non_sp500_lingos"]], axis=1)

In [29]:
df_submissions_daily.tail()

### 3.1.9. Save the submissions dataframe

In [30]:
df_submissions_daily.to_csv("data/feature_submissions.csv", sep=";", index=False)

## 3.2. Comments

### 3.2.1 Calculate sentiment per day

In [31]:
df_comments["body_polarity"] = 0.0
df_comments["body_subjectivity"] = 0.0
df_comments["emojis"] = ""

def get_comment_sentiment(body):
    nlp_body = nlp(str(body))
    emojis = nlp_body._.emoji
    emojis = [emoji[0] for emoji in emojis]
    return pd.Series([nlp_body._.polarity, nlp_body._.subjectivity, emojis])

df_comments[["body_polarity", "body_subjectivity", "emojis"]] = df_comments.apply(lambda row: get_comment_sentiment(row["body"]), axis=1)
df_comments.head()

,author_x,body,created_utc_x,id_x,score_x,parent_id,body_polarity,body_subjectivity,emojis
0,ADONIS_VON_MEGADONG,I already got in but decided to say YOLO and s...,2021-01-28,gl35t1p,3,l6x2gv,0.000000,0.0,[]
1,ADONIS_VON_MEGADONG,Yeah the thing is I actually have some faith i...,2021-01-28,gl377lu,2,l6x2gv,0.066667,0.5,[]
2,sf9191,I’ve been with them for two years and have hat...,2021-01-28,gl389xu,1,l6x2hd,-0.900000,0.7,[]
3,amcneel,BB,2021-01-28,gl35j7i,6,l6x2hj,0.000000,0.0,[]
4,MineIsLongerThanYour,Not now man. Focus on the fucking gme,2021-01-28,gl35kwp,3,l6x2ho,-0.600000,0.8,[]


In [32]:
# Group by date
df_comments["body_polarity"] = df_comments["body_polarity"].fillna(0)
df_comments["body_polarity"] = df_comments["body_polarity"].astype(float)
df_comments_body_polarity = df_comments.groupby(["created_utc_x"])["body_polarity"].sum().reset_index()

df_comments["body_subjectivity"] = df_comments["body_subjectivity"].fillna(0)
df_comments["body_subjectivity"] = df_comments["body_subjectivity"].astype(float)
df_comments_body_subjectivity = df_comments.groupby(["created_utc_x"])["body_subjectivity"].sum().reset_index()

### 3.2.2. Calculate emojis per day

In [33]:
df_comments_emojis = df_comments.groupby(["created_utc_x"]).apply(lambda x: str(list(x["emojis"]))).to_frame("emojis").reset_index()
df_comments_emojis["emojis"] = df_comments_emojis["emojis"].apply(sanitize_emojis)
df_comments_emojis["emojis"] = df_comments_emojis.apply(lambda x: dict(Counter(x["emojis"])), axis=1)

### 3.2.2. Calculate mentioned stocks and lingos count per day

In [34]:
df_comments[["sp500_stocks", "non_sp500_stocks", "sp500_lingos", "non_sp500_lingos"]] = df_comments.apply(lambda x: calculate_mentioned_stocks(x["body"]), axis=1)
df_comments.head()

,author_x,body,created_utc_x,id_x,score_x,parent_id,body_polarity,body_subjectivity,emojis,sp500_stocks,non_sp500_stocks,sp500_lingos,non_sp500_lingos
0,ADONIS_VON_MEGADONG,I already got in but decided to say YOLO and s...,2021-01-28,gl35t1p,3,l6x2gv,0.000000,0.0,[],[],[],[],[]
1,ADONIS_VON_MEGADONG,Yeah the thing is I actually have some faith i...,2021-01-28,gl377lu,2,l6x2gv,0.066667,0.5,[],[],[],[],[]
2,sf9191,I’ve been with them for two years and have hat...,2021-01-28,gl389xu,1,l6x2hd,-0.900000,0.7,[],[],[],[],[]
3,amcneel,BB,2021-01-28,gl35j7i,6,l6x2hj,0.000000,0.0,[],[],[],[],[]
4,MineIsLongerThanYour,Not now man. Focus on the fucking gme,2021-01-28,gl35kwp,3,l6x2ho,-0.600000,0.8,[],[],[],[],[]


In [35]:
# Group by date
df_comments_sp500_stocks = df_comments.groupby(["created_utc_x"])["sp500_stocks"].apply(",".join).reset_index()
df_comments_non_sp500_stocks = df_comments.groupby(["created_utc_x"])["non_sp500_stocks"].apply(",".join).reset_index()

df_comments_sp500_lingos = df_comments.groupby(["created_utc_x"])["sp500_lingos"].apply(",".join).reset_index()
df_comments_non_sp500_lingos = df_comments.groupby(["created_utc_x"])["non_sp500_lingos"].apply(",".join).reset_index()

In [36]:
# Sanitize data
df_comments_sp500_stocks["sp500_stocks"] = df_comments_sp500_stocks["sp500_stocks"].apply(sanitize_column)
df_comments_non_sp500_stocks["non_sp500_stocks"] = df_comments_non_sp500_stocks["non_sp500_stocks"].apply(sanitize_column)

df_comments_sp500_lingos["sp500_lingos"] = df_comments_sp500_lingos["sp500_lingos"].apply(sanitize_column)
df_comments_non_sp500_lingos["non_sp500_lingos"] = df_comments_non_sp500_lingos["non_sp500_lingos"].apply(sanitize_column)

In [37]:
# Count stocks and lingos
df_comments_sp500_stocks["sp500_stocks"] = df_comments_sp500_stocks["sp500_stocks"].str.strip("[]").str.split(",")
df_comments_sp500_stocks["sp500_stocks"] = df_comments_sp500_stocks.apply(lambda x: dict(Counter(x['sp500_stocks'])), axis=1)

df_comments_non_sp500_stocks["non_sp500_stocks"] = df_comments_non_sp500_stocks["non_sp500_stocks"].str.strip("[]").str.split(",")
df_comments_non_sp500_stocks["non_sp500_stocks"] = df_comments_non_sp500_stocks.apply(lambda x: dict(Counter(x["non_sp500_stocks"])), axis=1)

df_comments_sp500_lingos["sp500_lingos"] = df_comments_sp500_lingos["sp500_lingos"].str.strip("[]").str.split(",")
df_comments_sp500_lingos["sp500_lingos"] = df_comments_sp500_lingos.apply(lambda x: dict(Counter(x["sp500_lingos"])), axis=1)

df_comments_non_sp500_lingos["non_sp500_lingos"] = df_comments_non_sp500_lingos["non_sp500_lingos"].str.strip("[]").str.split(",")
df_comments_non_sp500_lingos["non_sp500_lingos"] = df_comments_non_sp500_lingos.apply(lambda x: dict(Counter(x["non_sp500_lingos"])), axis=1)

### 3.2.3. Count  per day

In [38]:
df_comments_count = df_comments.groupby(["created_utc_x"])["body"].count().reset_index()
df_comments_count.rename(columns={"body": "count"}, inplace=True)

### 3.2.4. Calculate total scores per day

In [39]:
df_comments["score_x"] = df_comments["score_x"].fillna(0)
df_comments["score_x"] = df_comments["score_x"].astype(int)
df_comments_score = df_comments.groupby(["created_utc_x"])["score_x"].sum().reset_index()

### 3.2.5. Create the final comments dataframe

In [40]:
df_comments_daily = pd.concat([df_comments_sp500_stocks["created_utc_x"], 
                               df_comments_body_polarity["body_polarity"], 
                               df_comments_body_subjectivity["body_subjectivity"], 
                               df_comments_count["count"],
                               df_comments_score["score_x"],
                               df_comments_emojis["emojis"],
                               df_comments_sp500_stocks["sp500_stocks"], 
                               df_comments_non_sp500_stocks["non_sp500_stocks"],
                               df_comments_sp500_lingos["sp500_lingos"], 
                               df_comments_non_sp500_lingos["non_sp500_lingos"]], axis=1)

In [41]:
df_comments_daily.head()

,created_utc_x,body_polarity,body_subjectivity,count,score_x,emojis,sp500_stocks,non_sp500_stocks,sp500_lingos,non_sp500_lingos
0,2021-01-28,11215.636037,151064.552591,481439,7307273,"{'🤬': 130, '💎': 44029, '🚀': 129555, '😍': 151, ...","{''R'': 184, ''GME'': 38546, ''PM'': 290, ''BB...","{''SNDL'': 742, ''STAY'': 349, ''ON'': 2495, '...","{''BUY'': 2342, ''squeeze'': 937, ''buying'': ...","{''moon'': 146, ''BUY'': 2021, ' 'buying'': 68..."
1,2021-01-29,24279.126195,234495.826707,714325,11031800,"{'🚀': 206558, '🦅': 25, '💎': 107307, '✋': 13378...","{''DD'': 1237, ''GME'': 51536, ''CA'': 41, ''A...","{''NEXT'': 1318, ''SNDL'': 306, ' 'EM'': 73, '...","{''DD'': 1275, ''buying'': 3092, ''BUYING'': 5...","{''GO'': 3809, ''go'': 419, ' 'squeeze'': 391,..."
2,2021-01-30,21378.698455,139558.128577,400095,5214224,"{'🤷': 569, '🏻': 3713, '\': 3133, 'u': 2911, '2...","{''M'': 525, ''GME'': 34083, ''CCL'': 35, ''SC...","{''AAL'': 119, ''CMPS'': 1, ''UK'': 907, ''EDI...","{''moon'': 1444, ''buying'': 2181, ''DD'': 115...","{''BUY'': 282, ''buying'': 260, ''squeeze'': 2..."
3,2021-01-31,13794.394890,93609.909582,267732,3151759,"{'💎': 26882, '🙌': 9807, '🅱': 134, '️': 3030, '...","{''GME'': 25269, ''DO'': 602, ' 'K'': 278, ' '...","{''UK'': 406, ''IDEX'': 9, ''ON'': 491, ''MRVL...","{''holding'': 1067, ''BUY'': 626, ''buying'': ...","{''BUYING'': 90, ''YOLO'': 18, ''GO'': 366, ''..."
4,2021-02-01,12907.351098,109175.041330,335178,4764767,"{'🚀': 80806, '👌': 216, '☀': 33, '️': 2472, '💎'...","{''GME'': 34590, ' 'PM'': 69, ''BBBY'': 122, '...","{''FREE'': 100, ' 'UK'': 28, ''PLUG'': 38, ''L...","{''holding'': 1526, ''DD'': 960, ''BUY'': 1680...","{''go'': 275, ' 'squeeze'': 49, ''BUY'': 699, ..."


### 3.2.6. Save the comments dataframe

In [42]:
df_comments_daily.to_csv(feature_comments_csv, sep=";", index=False)

In [ ]:
# df_comments.drop([4929153, 4929185, 4929199, 4929235, 4929244, 5015671, 5015833, 5016105, 5021356], inplace=True)